In [2]:
import os
import openai
import backoff
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import torch
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics import classification_report
from peft import PeftModel

from Eval_utils import *

df_test = pd.read_csv("../Dataset/EconNLI_test.csv" )
df_test = df_test[df_test['label']==1]
df_test

,wiki_page,sentence,cause,effect,label
0,Exchange rate,"in our example, the forward exchange rate of ...",The forward exchange rate of the dollar buys f...,The forward exchange rate of the dollar is at...,1
1,Bond (finance),"this creates reinvestment risk, meaning the i...",Interest rates are falling,The investor is forced to find a new place fo...,1
3,Stock valuation,"according to the peg ratio, stock a is a bett...",Stock A has a lower PEG ratio,stock A is a better purchase,1
4,Productivity,"at the national level, productivity growth rai...",more real income improves people's ability to ...,productivity growth raises living standards,1
6,Demand shock,"when demand for goods or services increases, ...",The demand curve shifts to the right,The price (or price levels) increases,1
...,...,...,...,...,...
1017,Interest rate,at this zero lower bound the central bank fac...,market interest rates cannot realistically be ...,central bank faces difficulties with conventi...,1
1019,Bond (finance),"these are known as hybrid securities, **BECAU...",hybrid securities combine equity and debt feat...,they are known as hybrid securities,1
1020,Inflation,higher interest rates reduce the economy's mo...,Fewer people seek loans,Higher interest rates reduce the economy's mo...,1
1021,Unemployment,voluntary unemployment is attributed to the i...,"The socio-economic environment, including the ...",Involuntary unemployment exists,1


### ChatGPT/GPT4

In [ ]:
prompt = "If the event {} happens, what event will be caused? Answer the possible resulting event very briefly in one sentence. answer:"

prompt2 = "I will give you two economical events. Determine the connection between these two events, \
choosing from 'entailment', 'contradiction', and 'irrelevant'. \
'Entailment' means the two events describe the similar economical phenomena, \
'contradiction' means the two events describe the opposite economical phenomena, \
'irrelevant' means the two events are not related or not belonging to one of the above two classes. \
Only provide the label.\
Event 1: {}, event 2:{}"

for MODEL_NAME in ["ChatGPT","GPT4"]:

    hallucination_answers = []
    ent_cnt, con_cnt, irr_cnt = 0,0,0

    for index in tqdm(df_test.index):
        topic = df_test.iloc[index]["wiki_page"]
        premise = df_test.iloc[index]["cause"]
        hypothesis = df_test.iloc[index]["effect"]
        label = df_test.iloc[index]["label"]
        
        if MODEL_NAME == "ChatGPT":
            model_answer = prompt_chatgpt_with_backoff(prompt.format(premise)) 
        elif MODEL_NAME == "GPT4":
            model_answer = prompt_gpt4_with_backoff(prompt.format(premise)) 

        hallucination_clf = prompt_gpt4_with_backoff(prompt2.format(hypothesis,model_answer))
        hallucination_answers.append({"row_index":index, "hallucination_answer":hallucination_clf})

        print("premise: {}, hypothesis: {}, label: {}".format(premise, hypothesis, label))
        print("model_answer:{}".format(model_answer))
        print("whether hallucination?", hallucination_clf)
        print("=========================================================")
    

    for item in hallucination_answers:
        if "entailment" in item['hallucination_answer'].lower():
            ent_cnt+=1
        elif "contradiction" in item['hallucination_answer'].lower():
            con_cnt+=1
        elif "irrelevant" in item['hallucination_answer'].lower():
            irr_cnt+=1

    print("ent_cnt: ", ent_cnt)
    print("con_cnt: ", con_cnt)
    print("irr_cnt: ", irr_cnt)
    print("total: ", ent_cnt+con_cnt+irr_cnt)
    
    with open("results/hallucination_detection.txt", "a") as f:
        f.write(MODEL_NAME+"\n")
        f.write("ent_cnt: {} \n".format(ent_cnt))
        f.write("con_cnt: {} \n".format(con_cnt))
        f.write("irr_cnt: {} \n".format(irr_cnt))
        f.write("total: {} \n".format(ent_cnt+con_cnt+irr_cnt))

### llama

In [ ]:

# Remove comments on your desired model

# LLAMA2-7B-chat
# model_name = "../llama/Llama-2-7b-chat-hf"
# model = AutoModelForCausalLM.from_pretrained(model_name,
#     device_map="auto",
#     max_memory={2:"24GB",3:"24GB"}, 
#     torch_dtype=torch.float16,
#     )
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token

# LLAMA2-13B-chat
# model_name = "../llama/Llama-2-13b-chat-hf"
# model = AutoModelForCausalLM.from_pretrained(model_name,
#     device_map="auto",
#     max_memory={2:"24GB",3:"24GB"}, 
#     torch_dtype=torch.float16,
#     )
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token

# FINMA
# model_name = "ChanceFocus/finma-7b-nlp"
# model = AutoModelForCausalLM.from_pretrained(model_name,
#     device_map="auto",
#     max_memory={0:"24GB"}, 
#     torch_dtype=torch.float16
#     )
# tokenizer = AutoTokenizer.from_pretrained(model_name,unk_token ="<s>")
# tokenizer.pad_token = tokenizer.eos_token

# #Alpaca
# model_name = "../llama/alpaca-7b/"
# model = AutoModelForCausalLM.from_pretrained(model_name,
#     device_map="auto",
#     max_memory={0:"24GB",1:"24GB"}, 
#     torch_dtype=torch.float16,
#     )
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token


prompt = "If the event {} happens, what event will be caused? Answer the possible resulting event very briefly in one sentence. answer:"

prompt2 = "I will give you two economical events. Determine the connection between these two events, \
choosing from 'entailment', 'contradiction', and 'irrelevant'. \
'Entailment' means the two events describe the similar economical phenomena, \
'contradiction' means the two events describe the opposite economical phenomena, \
'irrelevant' means the two events are not related or not belonging to one of the above two classes. \
Only provide the label.\
Event 1: {}, event 2:{}"

hallucination_answers = []

for index in tqdm(df_test.index):
    topic = df_test.iloc[index]["wiki_page"]
    premise = df_test.iloc[index]["cause"]
    hypothesis = df_test.iloc[index]["effect"]
    label = df_test.iloc[index]["label"]
    # print("prompt: {}".format(prompt.format(premise, hypothesis)))
    model_answer = prompt_llama_like_model(prompt.format(premise),model,tokenizer)  
    prediction = model_answer.split("answer:")[1].strip()
    
    hallucination_clf = prompt_gpt4_with_backoff(prompt2.format(hypothesis,prediction))
    hallucination_answers.append({"row_index":index, "hallucination_answer":hallucination_clf})
    
    print("premise: {}, hypothesis: {}, label: {}".format(premise, hypothesis, label))
    print("model_answer:{}".format(model_answer))
    print("prediction: {}".format(prediction))
    print("whether hallucination?", hallucination_clf)
    print("=========================================================")

ent_cnt, con_cnt, irr_cnt = 0,0,0

for item in hallucination_answers:
    if "entailment" in item['hallucination_answer'].lower():
        ent_cnt+=1
    elif "contradiction" in item['hallucination_answer'].lower():
        con_cnt+=1
    elif "irrelevant" in item['hallucination_answer'].lower():
        irr_cnt+=1
        
print("ent_cnt: ", ent_cnt)
print("con_cnt: ", con_cnt)
print("irr_cnt: ", irr_cnt)
print("total: ", ent_cnt+con_cnt+irr_cnt)